In [204]:
import spacy as sp
from spacy.matcher import Matcher

import pandas as pd

In [205]:
data = pd.read_csv("test_data.csv")
nlp = sp.load("ru_core_news_md")

In [206]:
data.head()

,dlg_id,line_n,role,text
0,0,0,client,Алло
1,0,1,manager,Алло здравствуйте
2,0,2,client,Добрый день
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...
4,0,4,client,Ага


In [207]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 480 entries, 0 to 479
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   dlg_id  480 non-null    int64 
 1   line_n  480 non-null    int64 
 2   role    480 non-null    object
 3   text    480 non-null    object
dtypes: int64(2), object(2)
memory usage: 15.1+ KB


a) Извлекать реплики с приветствием – где менеджер поздоровался.

In [208]:
df = data[data["role"] == 'manager']

In [210]:
matcher = Matcher(nlp.vocab)
patterns = [
    [{"LOWER": "здравствуйте"}],
    [{"LEMMA": "добрый"}, {"LEMMA": {"REGEX": "(утро|день|вечер)"}}],
]
matcher.add("приветствие", patterns)

In [212]:
df_greeting = pd.DataFrame(data=None, columns=df.columns)
for i, line in enumerate(df["text"]):
    doc = nlp(line)
    greetings = matcher(doc)
    for _ in greetings:
        df_greeting = pd.concat([df_greeting, df.iloc[[i]]])
df_greeting

,dlg_id,line_n,role,text
1,0,1,manager,Алло здравствуйте
110,1,1,manager,Алло здравствуйте
166,2,2,manager,Алло здравствуйте
250,3,1,manager,Алло дмитрий добрый день


b) Извлекать реплики, где менеджер представил себя. 

In [213]:
df = data[data["role"] == 'manager']

In [214]:
matcher = Matcher(nlp.vocab)

In [215]:
patterns = [
    [{"LOWER": "меня"}, {"POS": {"REGEX": "(PERSON|PROPN)"}, "OP": "?"}, {"LOWER": "зовут"}],
    [{"LOWER": "звать"}, {"LOWER": "меня"}],
    [{"LOWER": "я"}, {"POS": {"REGEX": "(PROPN|PERSON)"}}],
    [{"LOWER": {"REGEX": ("да|нет")}}, {"LOWER": "это"}, {"POS": {"REGEX": "(PROPN|PERSON)"}}]
]

In [216]:
matcher.add("представление", patterns)

In [217]:
df_performance = pd.DataFrame(data=None, columns=df.columns)
for i, line in enumerate(df["text"]):
    doc = nlp(line)
    performances = matcher(doc)
    for _ in performances:
        df_performance = pd.concat([df_performance, df.iloc[[i]]])
df_performance

,dlg_id,line_n,role,text
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...
111,1,2,manager,Меня зовут ангелина компания диджитал бизнес з...
167,2,3,manager,Меня зовут ангелина компания диджитал бизнес з...
251,3,2,manager,Добрый меня максим зовут компания китобизнес у...
338,5,1,manager,Да это анастасия


c) Извлекать имя менеджера. 

In [218]:
arr_name = ["NULL" for _ in range(len(df_performance["text"]))]
for i, line in enumerate(df_performance["text"]):
    sen = nlp(line)
    for name in sen:
        if name.pos_ == "PROPN" and name.text != "звоню":
            arr_name[i] = name
df_performance["name"] = arr_name
df_performance

,dlg_id,line_n,role,text,name
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...,ангелина
111,1,2,manager,Меня зовут ангелина компания диджитал бизнес з...,ангелина
167,2,3,manager,Меня зовут ангелина компания диджитал бизнес з...,ангелина
251,3,2,manager,Добрый меня максим зовут компания китобизнес у...,максим
338,5,1,manager,Да это анастасия,анастасия


d) Извлекать название компании.

In [219]:
import random
from spacy.training import Example

TRAINING_DATA = [
    ("Добрый меня максим зовут компания китобизнес удобно говорить", 
    {"entities": [(34,44,"COMPANY")]}),
    ("Меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления а мы сели обратила внимание что у вас срок заканчивается", 
    {"entities": [(29,44,"COMPANY")]}),
]

for i in range(20):
    random.shuffle(TRAINING_DATA)
    for batch in sp.util.minibatch(TRAINING_DATA, size=4):
        for text, annotations in batch:
            doc = nlp.make_doc(text)
            example = Example.from_dict(doc, annotations)
            nlp.update([example])
        
nlp.to_disk("model")

In [220]:
nlp = sp.load("model")
df_company = pd.DataFrame(data=None, columns=df.columns)
arr_company = []
for i, line in enumerate(df["text"]):
    doc = nlp(line)
    for entity in doc.ents:
        if entity.label_ == "COMPANY":
            df_company = pd.concat([df_company, df.iloc[[i]]])
            arr_company.append(entity.text)
            
df_company["company"] = arr_company
df_company

,dlg_id,line_n,role,text,company
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...,диджитал бизнес
111,1,2,manager,Меня зовут ангелина компания диджитал бизнес з...,диджитал бизнес
167,2,3,manager,Меня зовут ангелина компания диджитал бизнес з...,диджитал бизнес
251,3,2,manager,Добрый меня максим зовут компания китобизнес у...,китобизнес


e) Извлекать реплики, где менеджер попрощался.

In [222]:
nlp = sp.load("ru_core_news_md")

In [233]:
matcher = Matcher(nlp.vocab)
pattern = [
    [{"LOWER": "до"}, {"LOWER": {"REGEX": ("свидания|встречи?|скорого")}}],
    [{"LOWER": "всего"}, {"LOWER": {"REGEX": ("хорошего|доброго")}}]
]
matcher.add("прощание", pattern)

In [283]:
df_parting = pd.DataFrame(data=None, columns=df.columns)
for i, line in enumerate(df["text"]):
    doc = nlp(line)
    partings = matcher(doc)
    for _ in partings:
        df_parting = pd.concat([df_parting, df.iloc[[i]]])
df_parting = df_parting.drop_duplicates(keep='first')
df_parting

,dlg_id,line_n,role,text
108,0,108,manager,Всего хорошего до свидания
162,1,53,manager,Угу да вижу я эту почту хорошо тогда исправлю ...
163,1,54,manager,До свидания
300,3,51,manager,Угу все хорошо да понедельника тогда всего доб...
335,4,33,manager,Во вторник все ну с вами да тогда до вторника ...
479,5,142,manager,Ну до свидания хорошего вечера


f) Проверять требование к менеджеру: «В каждом диалоге обязательно необходимо поздороваться и попрощаться с клиентом»

In [284]:
dlg_id = list(set(df["dlg_id"]))
df_manager = pd.DataFrame(data = [[dlg_id[i], 0] for i in range(len(dlg_id))], 
                          columns = ["dlg_id","Fulfillment of requirements"])

for i in dlg_id:
    if i in df_parting['dlg_id'].values and i in df_performance['dlg_id'].values:
        df_manager.at[i, "Fulfillment of requirements"] = 1

df_manager

,dlg_id,Fulfillment of requirements
0,0,1
1,1,1
2,2,0
3,3,1
4,4,0
5,5,1
